# Heatmap Overlay from Patch Predictions

Aggregate patch-level scores to a slide-level heatmap and overlay on the thumbnail.

In [ ]:
# Bootstrap shared utils and ensure a demo WSI if needed
import sys
from pathlib import Path
root = Path.cwd()
shared = root / 'shared'
if not shared.exists():
    alt = root.parent / 'shared'
    if alt.exists():
        shared = alt
if str(shared) not in sys.path:
    sys.path.insert(0, str(shared))
from utils import get_notebook_config, ensure_demo_wsi
CFG = get_notebook_config()
DATA_DIR = Path(CFG['data_dir'])
RESULTS_DIR = Path(CFG['results_dir'])
DATA_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
WSI_PATH = ensure_demo_wsi(DATA_DIR)
print(f"WSI_PATH = {WSI_PATH}")

In [ ]:
import sys, os
from pathlib import Path
try:
    from shared import utils as u
except ImportError:
    repo_url = "https://github.com/anand-indx/dp-t25.git"; dest = "/content/dp-t25"
    if 'google.colab' in sys.modules and not os.path.exists(dest):
        import subprocess
        subprocess.run(['git', 'clone', '--depth', '1', repo_url, dest], check=False)
        sys.path.insert(0, dest)
    else:
        sys.path.insert(0, str(Path.cwd().parents[1]))
    from shared import utils as u

import numpy as np, matplotlib.pyplot as plt
from PIL import Image
from openslide import OpenSlide, OpenSlideError

DATA_DIR = u.get_data_dir()
WSI_PATH = u.get_wsi_path(DATA_DIR)

# Try to use real thumbnail if WSI exists; fallback to synthetic
thumb = None
try:
    slide = OpenSlide(str(WSI_PATH))
    thumb = slide.get_thumbnail((1024, 1024)).convert('RGB')
except Exception:
    thumb = Image.new('RGB', (1024, 1024), 'white')

# Load scores from patches if present, else synthetic
patch_dir = DATA_DIR / 'patches'
if patch_dir.exists():
    tiles = list(patch_dir.glob('*.png'))
    if len(tiles) > 0:
        grid_size = (32, 32)
        scores = np.zeros(grid_size, dtype=float)
        for i, t in enumerate(tiles[: grid_size[0]*grid_size[1]]):
            r = i // grid_size[1]
            c = i % grid_size[1]
            scores[r, c] = 1.0
    else:
        scores = np.random.rand(32, 32)
else:
    scores = np.random.rand(32, 32)

plt.figure(figsize=(6,6))
plt.imshow(thumb)
plt.imshow(scores, cmap='jet', alpha=0.4, extent=[0,1024,1024,0])
plt.axis('off')
plt.show()